# Politecnico Malvinas Argentinas- PDH: Preprocesamiento, Normalización y Técnicas Básicas

**Estas líneas instalan las librerías que vamos a necesitar para trabajar con el texto. Algunas nos ayudan a limpiar, otras a analizar, otras a comparar palabras. Se recomienda usarlas si estás trabajando en Google Colab o un entorno nuevo.   
Importamos las funciones que vamos a usar. Algunas sirven para dividir el texto en palabras, otras para eliminar lo que no sirve, y otras para comparar o transformar el texto en números.**

In [25]:
# Instalación de librerías necesarias (si es necesario en Colab)
!pip install nltk spacy scikit-learn thefuzz stop_words
!pip install fuzzywuzzy python-Levenshtein
!pip install pandas

import re
import nltk
import spacy
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.5 MB 2.6 MB/s eta 0:00:05
   --- ------------------------------------ 1.0/11.5 MB 2.4 MB/s eta 0:00:05
   ----- ---------------------------------- 1.6/11.5 MB 2.3 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.5 MB 2.3 MB/s eta 0:00:05
   --------- ------------------------------ 2.6/11.5 MB 2.4 MB/s eta 0:00:04
   ---------- ----------------------------- 3.1/11.5 MB 2.3 MB/s eta 0:00:04
   ------------ --------------------------- 3.7/11.5 MB 2.4 MB/s eta 0:00:04
   -------------- ------------------------- 4.2/11.5 MB 2.4 MB/s eta 0:00:04
   ---------------- ----------------------- 4.7/11.5 MB 2.4 MB/s eta 0:00:03
   ------------------ --------------------- 5.2/11.5 MB 2.4 MB/s eta 0:00:03
   -------------------- ------------------- 5.8/11.5 MB 2.3 MB/s eta 0:00:03
   ----------

**Estas descargas son necesarias para que NLTK pueda dividir textos en palabras y eliminar palabras vacías como "el", "la", "de", etc.**

In [26]:
# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Esto descarga un modelo en español que spaCy necesita para entender el texto: saber qué es un sustantivo, un verbo, reconocer nombres de lugares, personas, etc.**

In [19]:
# Cargar modelo en español de spaCy
!python -m spacy download es_core_news_md
nlp = spacy.load("es_core_news_md")

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     ---------------------------------------- 0.3/42.3 MB ? eta -:--:--
     -- ------------------------------------- 2.6/42.3 MB 10.0 MB/s eta 0:00:04
     ----- ---------------------------------- 5.8/42.3 MB 12.2 MB/s eta 0:00:04
     -------- ------------------------------- 8.9/42.3 MB 13.2 MB/s eta 0:00:03
     ------------ -------------------------- 13.4/42.3 MB 14.7 MB/s eta 0:00:02
     ----------------- --------------------- 19.1/42.3 MB 16.5 MB/s eta 0:00:02
     ---------------------- ---------------- 23.9/42.3 MB 17.3 MB/s eta 0:00:02
     --------------------------- ----------- 29.4/42.3 MB 18.4 MB/s eta 0:00:01
     ------------------------------ -------- 32.8/42.3 MB 18.9 MB/s eta 0:00:01
     ---------------------------------- ---- 37.5/42.3 MB 18.2 MB/s eta 0:00:01
     ------------------------------------- - 41.2/42.3 MB 18.0 MB/s eta 0:00:01
     --------------------------------------  42.2/42.3 M

# 1. Preprocesamiento de texto

# Tokenizacion

Quizás una de las operaciones más básicas que podemos hacer es contar palabras. Imaginemos un caso extremadamente sencillo relacionado al aprendizaje automático. Pensemos en un clasificador de positividad / negatividad.

Podríamos usar una regla como:

si "bueno" está en el texto, clasificamos como positivo.

Claramente un enfoque tan sencillo va a ser propenso a muchos errores. Iremos viendo cómo aplicarlo y mejorarlo.

La operación de separar los textos en unidades básicas (o tokens) la llamamos tokenización.

**Este código simplemente divide la frase en palabras, usando el espacio como separador. Es una forma muy básica de tokenizar el texto (aunque menos precisa que word_tokenize o spaCy), pero útil para entender cómo se descompone una oración.**

In [27]:
doc2 = 'Usualmente, existe una relación costo-beneficio entre las distintas técnicas.'
doc2.split(' ')

['Usualmente,',
 'existe',
 'una',
 'relación',
 'costo-beneficio',
 'entre',
 'las',
 'distintas',
 'técnicas.']

Vemos que en castellano, el lenguaje natural (escrito) puede tener distintas fuentes de variaciones:

Mayúsculas
Tildes
Signos ortográficos
Errores de tipeo
Variaciones propias del lenguaje como la conjugación
Artefactos de la escritura informal como "holaaa"
¿Cómo abordarán esto nuestros algorítmos?

Una solución es la normalización: con acepción similar aunque distinta a la que usamos en estadística, va a significar transformar a los strings a representaciones iguales.

Podemos:

Sacar espacios
Pasar a minúsculas
Quitar tildes

**Este bloque instala la librería unidecode, que sirve para eliminar acentos y caracteres especiales de las palabras. El comando %%capture se usa para ocultar los mensajes de instalación, ideal si no querés que se llene la consola de texto mientras corrés el notebook.**

In [28]:
%%capture
!pip install unidecode

**Convierte las palabras a minúsculas y les saca los acentos. Así “técnicas” pasa a “tecnicas”. Esto ayuda a comparar palabras sin errores por tildes.**

In [29]:
import unidecode
[unidecode.unidecode(w.lower().strip()) for w in doc2.split(' ')]

['usualmente,',
 'existe',
 'una',
 'relacion',
 'costo-beneficio',
 'entre',
 'las',
 'distintas',
 'tecnicas.']

Otras normalizaciones más sofisticadas serían:
- Pasar de plural a singular
- Convertir el género de la palabra
- Conjugar los verbos en infinitivo

¡Es de esperarse que esto requiera métodos más complejos!

Vemos que con la tokenización anterior, todavía tenemos palabras con signos de puntuación pegados. Armar nuestro vocabulario es un paso clave para nuestros modelos de *Machine Learning*. 

**Convierte todo a minúsculas y separa el texto en palabras. Es el primer paso para analizar el contenido palabra por palabra.**

In [32]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import nltk
nltk.download('punkt_tab')




text = "¡Hola! ¿Cómo estás? Estoy aprendiendo NLP con spaCy y NLTK. NLP es fascinante."

# Tokenización y Normalización
words = word_tokenize(text.lower())
print("Tokenización y Normalización:", words)




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


Tokenización y Normalización: ['¡hola', '!', '¿cómo', 'estás', '?', 'estoy', 'aprendiendo', 'nlp', 'con', 'spacy', 'y', 'nltk', '.', 'nlp', 'es', 'fascinante', '.']


**Elimina signos de puntuación y caracteres raros. Deja solo letras y espacios para que el texto esté limpio y más fácil de analizar.**

In [33]:
# Expresiones Regulares
text_clean = re.sub(r'[^a-zA-Záéíóúüñ ]', '', text.lower())
print("Texto limpio con regex:", text_clean)



Texto limpio con regex: hola cómo estás estoy aprendiendo nlp con spacy y nltk nlp es fascinante


**Quita palabras muy comunes como “el”, “la”, “y”, que no aportan mucho significado. Así nos enfocamos en las palabras importantes del texto.**

In [34]:
# Stop Words
stop_words = set(stopwords.words('spanish'))
filtered_words = [word for word in words if word not in stop_words]
print("Sin Stop Words:", filtered_words)

Sin Stop Words: ['¡hola', '!', '¿cómo', '?', 'aprendiendo', 'nlp', 'spacy', 'nltk', '.', 'nlp', 'fascinante', '.']


# 2. Técnicas de NLP con NLTK y spaCy

**Reduce las palabras a su raíz, por ejemplo, “comiendo” se convierte en “com”. Esto ayuda a agrupar variaciones de una palabra en una sola.**

In [35]:
# Stemming con NLTK
stemmer = SnowballStemmer("spanish")
stemmed_words = [stemmer.stem(word) for word in filtered_words]
print("Stemming:", stemmed_words)


Stemming: ['¡hol', '!', '¿com', '?', 'aprend', 'nlp', 'spacy', 'nltk', '.', 'nlp', 'fascin', '.']


**La lematización convierte una palabra a su forma base (o lema). Por ejemplo, “jugando” se convierte en “jugar”. Esto ayuda a estandarizar las palabras para que el análisis sea más preciso.**

In [36]:
# Lemmatización con spaCy
doc = nlp(" ".join(filtered_words))
lemmas = [token.lemma_ for token in doc]
print("Lemmatización:", lemmas)

Lemmatización: ['¡', 'hola', '!', '¿', 'cómo', '?', 'aprender', 'nlp', 'spacy', 'nltk', '.', 'nlp', 'fascinante', '.']


**Este bloque calcula la similitud entre dos palabras o textos. Por ejemplo, la distancia fuzzy entre “casa” y “casas” mide cuán similares son. Esto se usa para comparar textos y detectar similitudes.**

In [37]:
# Distancias entre strings
print("Distancia fuzzy entre 'casa' y 'casas':", fuzz.ratio("casa", "casas"))
texts = ["El gato come pescado", "El perro ladra en la casa"]

Distancia fuzzy entre 'casa' y 'casas': 89


**Este bloque instala una librería llamada pyjarowinkler que se usa para calcular la similitud entre cadenas de texto con el algoritmo Jaro-Winkler. También se importa edit_distance de nltk para calcular la distancia de Levenshtein entre las palabras. Estas técnicas son útiles para comparar cómo se parecen dos cadenas de texto.**

In [38]:
# segundo ejemplo %%capture
!pip install pyjarowinkler
from nltk.metrics import edit_distance
from pyjarowinkler import distance as jwdist

**Compara pares de palabras usando dos métricas:
Levenshtein: Cuenta las operaciones necesarias para convertir una palabra en otra.
Jaro-Winkler: Mide la similitud entre dos cadenas considerando coincidencias cercanas.**

In [39]:
# Definimos una lista de tuplas de palabras
palabras = [("pero", "perro"),("pero", "pierdo"), ("nueve", "mueve"),  ("totalmente","diferentes"), ("pero", "basta")]

# Calculamos las metricas de distancia pasando cada tupla como argumentos a levdist() y get_jaro_distance()
for x,y in palabras:
    print(f"'{x}' vs '{y}':")
    print("Distancia Levenshtein ->", edit_distance(x,y))
    print("Similitud Jaro Winkler ->",jwdist.get_jaro_distance(x,y))
    print("-"*40)

'pero' vs 'perro':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.07
----------------------------------------
'pero' vs 'pierdo':
Distancia Levenshtein -> 2
Similitud Jaro Winkler -> 0.11
----------------------------------------
'nueve' vs 'mueve':
Distancia Levenshtein -> 1
Similitud Jaro Winkler -> 0.13
----------------------------------------
'totalmente' vs 'diferentes':
Distancia Levenshtein -> 7
Similitud Jaro Winkler -> 0.48
----------------------------------------
'pero' vs 'basta':
Distancia Levenshtein -> 5
Similitud Jaro Winkler -> 1.0
----------------------------------------


# 3. Vectorización y Representación del Texto

**Define una lista de frases o textos que se usarán para realizar análisis de texto,**

In [40]:
texts = ["El gato come pescado", "El perro ladra en la casa"]

**Calcula la representación TF-IDF de los textos. El TF-IDF es una técnica de representación de texto que mide la importancia de una palabra dentro de un conjunto de documentos. Este método es útil para la clasificación y búsqueda de información.**

In [42]:
# TF-IDF
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(texts)
pd.DataFrame(X_tfidf.toarray(), columns=vectorizer_tfidf.get_feature_names_out())

casa      come        el       en      gato       la    ladra    perro  \
0  0.00000  0.534046  0.379978  0.00000  0.534046  0.00000  0.00000  0.00000   
1  0.42616  0.000000  0.303216  0.42616  0.000000  0.42616  0.42616  0.42616   

    pescado  
0  0.534046  
1  0.000000

**Genera N-gramas a partir de los textos. Los N-gramas son secuencias de "n" palabras consecutivas que aparecen en el texto. En este caso, el código genera pares y triples de palabras (2-gramas y 3-gramas). Esto es útil para capturar patrones o contextos dentro del texto, como en el análisis de sentimientos o clasificación de texto.**

In [43]:
# N-gramas
vectorizer_ngram = CountVectorizer(ngram_range=(2,3), min_df=1)
X_ngram = vectorizer_ngram.fit_transform(texts)
pd.DataFrame(X_ngram.toarray(), columns=vectorizer_ngram.get_feature_names_out())


come pescado  el gato  el gato come  el perro  el perro ladra  en la  \
0             1        1             1         0               0      0   
1             0        0             0         1               1      1   

   en la casa  gato come  gato come pescado  la casa  ladra en  ladra en la  \
0           0          1                  1        0         0            0   
1           1          0                  0        1         1            1   

   perro ladra  perro ladra en  
0            0               0  
1            1               1

# 4. Tareas Avanzadas con spaCy

**Este código utiliza spaCy para realizar un análisis gramatical sobre la oración proporcionada. Al usar el modelo en español de spaCy, se identifica la función gramatical de cada palabra en la frase, como sustantivos, verbos, adjetivos, etc. El resultado es un análisis estructurado de la oración, que es clave para tareas como la extracción de información o la comprensión del texto.**

In [44]:
sentence = "El presidente de Argentina visitó Buenos Aires."
doc = nlp(sentence)


**Imprime las etiquetas de partes del discurso de cada palabra en la oración, como sustantivos, verbos, etc.
¿Para qué sirve?
Ayuda a entender la función gramatical de las palabras en un texto.**

In [45]:
# POS Tagging
print([(token.text, token.pos_) for token in doc])                           

[('El', 'DET'), ('presidente', 'NOUN'), ('de', 'ADP'), ('Argentina', 'PROPN'), ('visitó', 'VERB'), ('Buenos', 'PROPN'), ('Aires', 'PROPN'), ('.', 'PUNCT')]


**Extrae entidades nombradas del texto, como nombres de países, personas o lugares.
¿Para qué sirve?
Ayuda a identificar información clave dentro de un texto, útil en tareas como resumen automático o extracción de datos.**

In [46]:
# Named Entity Recognition (NER)
print([(ent.text, ent.label_) for ent in doc.ents])

[('Argentina', 'LOC'), ('Buenos Aires', 'LOC')]


**Muestra una visualización de las relaciones gramaticales entre las palabras de la oración, como sujeto, verbo, objeto, etc.
¿Para qué sirve?
Ayuda a entender la estructura de una oración y cómo se conectan sus partes, útil para análisis profundo del lenguaje.**

In [50]:
# Dependencias sintácticas
!pip install spacy
import spacy
from spacy import displacy

displacy.render(doc, style='dep', jupyter=True)

In [51]:

# Limitaciones de Bolsa de Palabras
print("BoW no captura el significado de las palabras ni el orden de las mismas.")

BoW no captura el significado de las palabras ni el orden de las mismas.


##  Propuesta de mejoras 
**Ahora realizaremos un aporte, con el fin de realizar una Limpieza avanzada del texto (normalización + eliminación de emojis)**


Algo muy importante hoy en día es procesar los textos **tal como se encuentran en la realidad**. Muchas veces vienen con **emojis, signos raros, acentos o palabras en mayúsculas**, especialmente en redes sociales, chats o comentarios de usuarios.
Todo eso puede dificultar el análisis, por eso es fundamental **limpiar y normalizar el texto antes de seguir**.
En primer lugar, vamos a hacer:
**Conversión a minúsculas**, para que todas las palabras estén uniformes.
**Eliminación de acentos**, usando la librería `unidecode`.
**Eliminación de emojis y caracteres especiales**, dejando solo letras y espacios.
Esto nos va a permitir trabajar con un texto mucho más limpio y listo para aplicar técnicas como tokenización, lematización o vectorización de forma más efectiva.

In [64]:
# Texto original con emojis, signos de exclamación y símbolos
text = "¡Hola! ¿Cómo estás? Estoy aprendiendo NLP 🤓 con spaCy y NLTK. NLP es 💡fascinante!!!"

# 1. Pasar todo a minúsculas y quitar acentos
text = unidecode.unidecode(text.lower())

# 2. Eliminar emojis y caracteres no alfabéticos (dejamos solo letras y espacios)
text_clean = re.sub(r'[^a-zñáéíóúü\s]', '', text)

print("Texto limpio y normalizado:", text_clean)

Texto limpio y normalizado: hola como estas estoy aprendiendo nlp  con spacy y nltk nlp es fascinante


## 🔤 Normalización del texto

Primero convertimos el texto a **minúsculas** y eliminamos los **acentos** para unificar la escritura de las palabras. Por ejemplo, “Estás” y “estas” pasarán a ser la misma palabra.  
Esto ayuda a evitar duplicados y mejora la precisión en los análisis posteriores.


In [65]:
# 1. Pasar todo a minúsculas y quitar acentos
text = unidecode.unidecode(text.lower())

## 🧽 Limpieza del texto: eliminación de emojis y símbolos

En este paso, limpiamos el texto eliminando todo lo que **no sean letras o espacios**.  
Esto incluye **emojis, signos de puntuación y otros símbolos** que pueden entorpecer el análisis.  
Dejamos el texto limpio y preparado para procesarlo mejor.

In [63]:
# 2. Eliminar emojis y caracteres no alfabéticos (dejamos solo letras y espacios)
text_clean = re.sub(r'[^a-zñáéíóúü\s]', '', text)
print("Texto limpio y normalizado:", text_clean)

Texto limpio y normalizado: hola como estas estoy aprendiendo nlp  con spacy y nltk nlp es fascinante


# Resumen del Trabajo Realizado

En este proyecto, nos centramos en procesar y limpiar textos para su posterior análisis mediante técnicas de procesamiento de lenguaje natural (NLP). El objetivo fue aplicar diversas técnicas de preprocesamiento, ajustar el texto y analizarlo de una manera eficiente.

## Técnicas Aplicadas:

1. **Tokenización y Normalización:** 
   - Utilizamos **NLTK** para dividir el texto en palabras (tokens) y convertimos todas las palabras a minúsculas para normalizar el texto.

2. **Eliminación de Stop Words:**
   - Filtramos palabras vacías (como "y", "de", "en") que no aportan valor al análisis.

3. **Stemming:**
   - Aplicamos **SnowballStemmer** de **NLTK** para reducir las palabras a su raíz (ej. “jugando” a “jug”).

4. **Lemmatización:**
   - Utilizamos **spaCy** para convertir las palabras a su forma base, por ejemplo, “corriendo” a “correr”.

5. **Eliminación de caracteres especiales y emojis:**
   - Implementamos una expresión regular que eliminó caracteres no alfabéticos y emojis, dejándonos solo palabras y espacios, para que el texto sea más limpio y estructurado.

6. **Cálculo de Distancias entre cadenas:**
   - Medimos la similitud entre palabras utilizando **distancias fuzzy** y otras métricas como **Levenshtein** y **Jaro-Winkler**.

7. **Representación del Texto:**
   - Usamos técnicas como **TF-IDF** y **N-gramas** para representar el
